<a href="https://colab.research.google.com/github/jlerapatty/TUGASSIC/blob/main/202304560049_Joaquin_Yasser_Fahd_Lerapatty_ProjectSIC_IMDBTOP1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama Dataset   : imdb top 1000

Sumber Dataset : https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

Dataset ini mencakup data penting dari IMDb seperti :

Title (Judul film/acara)

Genre

Release Year

Runtime

IMDB Rating

MetaScore

Gross earnings

Certificate (rating usia seperti PG-13, R, dll.)

Director dan Stars

Konteks Masalah :
Proyek ini bertujuan untuk memprediksi rating IMDb sebuah film berdasarkan genre, guna mengetahui genre apa saja yang cenderung mendapatkan rating tinggi dan membantu pengambilan keputusan dalam rekomendasi film.


Identifikasi variabel target : IMDB_Rating dan Genre

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Tentukan path file dataset Anda
file_path = 'imdb_top_1000.csv'

# Memuat dataset dari file CSV
try:
    df = pd.read_csv(file_path)
    print(f"Dataset '{file_path}' berhasil dimuat.")
    print(f"Dimensi dataset: {df.shape[0]} baris, {df.shape[1]} kolom")
except FileNotFoundError:
    print(f"Error: File '{file_path}' tidak ditemukan.")
    df = None # Set df ke None jika gagal
except Exception as e:
    print(f"Error lain saat memuat dataset: {e}")
    df = None

# Menampilkan 5 baris pertama jika berhasil dimuat
if df is not None:
    print("\n5 Baris Pertama Dataset:")
    # Mengatur agar semua kolom ditampilkan (opsional, berguna jika kolom banyak)
    pd.set_option('display.max_columns', None)
    print(df.head())


In [ ]:
if df is not None:
    # Menampilkan informasi dasar (tipe data, non-null counts, memory usage)
    print("\nInformasi Dasar Dataset (df.info()):")
    df.info()
else:
    print("Dataset belum dimuat.")

In [ ]:
if df is not None:
    # Menampilkan statistik deskriptif untuk kolom numerik
    print("\nStatistik Deskriptif (Numerik):")
    print(df.describe().T)  # .T untuk transpose agar lebih mudah dibaca

    # Menampilkan statistik deskriptif untuk kolom object/kategorikal
    print("\nStatistik Deskriptif (Kategorikal/Object):")
    print(df.describe(include=['object', 'category']).T)
else:
    print("Dataset belum dimuat.")


In [ ]:
if df is not None:
    # Menghitung jumlah nilai hilang per kolom
    missing_values = df.isnull().sum()

    if missing_values.sum() == 0:
        print("\nTidak ditemukan nilai hilang dalam dataset.")
    else:
        # Menghitung persentase nilai hilang per kolom
        missing_percentage = (missing_values / len(df)) * 100
        # Menggabungkan hasil ke dalam DataFrame
        missing_info = pd.DataFrame({
            'Jumlah Hilang': missing_values,
            'Persentase Hilang': missing_percentage
        })
        # Menampilkan hanya kolom yang memiliki nilai hilang
        print("\nInformasi Nilai Hilang:")
        print(missing_info[missing_info['Jumlah Hilang'] > 0].sort_values(by='Persentase Hilang', ascending=False))
else:
    print("Dataset belum dimuat.")


In [ ]:
if df is not None:
    try:
        print("\nMembuat histogram untuk fitur numerik...")
        ax = df.hist(figsize=(12, 10), bins=20)  # bins disesuaikan
        plt.tight_layout()
        plt.savefig("histogram.png")  # Simpan ke file
        plt.show()
    except Exception as e:
        print(f"Gagal membuat histogram: {e}")
else:
    print("Dataset belum dimuat.")


In [ ]:
if df is not None:
    categorical_cols = df.select_dtypes(include=['object']).columns
    print("\nMembuat countplot untuk fitur kategorikal...")
    for column in categorical_cols:
        try:
            plt.figure(figsize=(10, 6))  # Sesuaikan ukuran jika perlu
            # Mengambil 20 kategori teratas jika > 20 agar plot tidak terlalu ramai
            top_categories = df[column].value_counts().nlargest(20).index
            sns.countplot(data=df[df[column].isin(top_categories)], y=column,
                          order=top_categories, palette='viridis')
            plt.title(f'Distribusi {column} (Top 20 jika > 20)')
            plt.xlabel('Jumlah')
            plt.ylabel(column)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Gagal membuat countplot untuk {column}: {e}")
else:
    print("Dataset belum dimuat.")


In [ ]:
if df is not None and 'IMDB_Rating' in df.columns:
    print("\nMembuat visualisasi hubungan dengan variabel target...")

    # Hubungan Numerik vs Target (contoh: Age, CGPA)
    numerical_to_plot = ['Meta_score']
    for col in numerical_to_plot:
        if col in df.columns:
            plt.figure(figsize=(15, 15))
            sns.boxplot(data=df, x='IMDB_Rating', y=col)
            plt.title(f'Hubungan antara {col} dan IMDB_Rating')
            plt.show()

    # Hubungan Kategorikal vs Target (contoh: Genre)
categorical_to_plot = ['Genre']
for col in categorical_to_plot:
    if col in df.columns:
        # Pecah genre menjadi list lalu explode
        df_exploded = df.copy()
        df_exploded[col] = df_exploded[col].astype(str).str.split(', ')
        df_exploded = df_exploded.explode(col)  # Tiap genre jadi baris baru

        plt.figure(figsize=(20, 10))
        sns.countplot(data=df_exploded, x=col, hue='IMDB_Rating', palette='pastel')
        plt.title(f'Hubungan antara {col} dan IMDB_Rating')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()


In [ ]:
if df is not None and 'Runtime' in df.columns:
    # Cek nilai unik di kolom 'Runtime'
    print("Nilai unik di 'Runtime':", df['Runtime'].unique())

    # Coba konversi ke numerik, paksa error menjadi NaN
    df['Runtime'] = pd.to_numeric(df['Runtime'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')

    # Cek apakah ada NaN yang muncul setelah konversi (jika ada nilai non-numerik)
    if df['Runtime'].isnull().sum() > 0:
        print("Peringatan: Ada nilai non-numerik di 'Runtime', menghasilkan NaN.")

        # Handle NaN jika muncul (misal: imputasi median)
        median_fs = df['Runtime'].median()

        # Cek apakah median_fs adalah NaN
        if pd.isna(median_fs):
            print("Tidak ada nilai yang valid untuk menghitung median.")
        else:
            df['Runtime'] = df['Runtime'].fillna(median_fs)
            print(f"NaN di 'Runtime' diimputasi dengan median: {median_fs}")
    else:
        print("Kolom 'Runtime' berhasil dikonversi ke numerik.")

    # Verifikasi tipe data baru`
    print(f"Tipe data baru 'Runtime': {df['Runtime'].dtype}")
else:
    print("Dataset atau kolom 'Runtime' tidak ditemukan.")


In [ ]:
if df is not None:
    print("\\n--- Memulai Encoding Variabel Kategorikal ---")
    # Identifikasi kolom kategorikal (tipe 'object' setelah Financial Stress dikonversi)
    categorical_cols_to_encode = df.select_dtypes(include=['object', 'category']).columns.tolist()

    # Hapus kolom yang tidak relevan jika ada (misal: id jika tidak dihapus sebelumnya)
    if 'Gross' in categorical_cols_to_encode:
        categorical_cols_to_encode.remove('Gross')  # Contoh jika 'Gross' masih object

    if categorical_cols_to_encode:
        print(f"Kolom kategorikal yang akan di-encode: {categorical_cols_to_encode}")

        # Terapkan One-Hot Encoding
        df_encoded = pd.get_dummies(df, columns=categorical_cols_to_encode, drop_first=True, dtype=int)

        # Hapus kolom 'Gross' asli jika masih ada dan tidak diperlukan
        if 'Gross' in df_encoded.columns:
            df_encoded = df_encoded.drop(['Gross'], axis=1)
            print("Kolom 'Gross' telah dihapus.")

        print(f"Dimensi dataset setelah One-Hot Encoding: {df_encoded.shape}")
        print("Contoh beberapa nama kolom baru:")
        print(df_encoded.columns[:5].tolist(), "...")
    else:
        print("Tidak ada kolom kategorikal yang perlu di-encode.")
        df_encoded = df.copy()
        if 'Gross' in df_encoded.columns:  # Hapus Gross jika tidak ada encoding
            df_encoded = df_encoded.drop(['Gross'], axis=1)
            print("Kolom 'Gross' telah dihapus.")
else:
    print("Dataframe 'df' tidak tersedia.")
    df_encoded = None


In [ ]:
from sklearn.preprocessing import MinMaxScaler

if 'df_encoded' in locals() and df_encoded is not None:
    print("\\n--- Memulai Scaling Fitur Numerik ---")
    # Identifikasi fitur numerik ASLI (sebelum OHE) untuk di-scale
    original_numerical_cols = ['Released_Year','Runtime','Meta_Score','No_of_Votes']
    if 'IMDB_Rating' in original_numerical_cols:
        original_numerical_cols.remove('IMDB_Rating')
    numerical_features_to_scale = [col for col in original_numerical_cols if col in df_encoded.columns]

    if numerical_features_to_scale:
        print(f"Fitur numerik yang akan di-scale: {numerical_features_to_scale}")

        scaler = MinMaxScaler()
        df_scaled = df_encoded.copy()
        # Terapkan scaler (idealnya fit_transform di train, transform di test)
        df_scaled[numerical_features_to_scale] = scaler.fit_transform(df_scaled[numerical_features_to_scale])

        print("\\nContoh data setelah normalisasi (fitur numerik yang di-scale):")
        print(df_scaled[numerical_features_to_scale].head())
        print("\\nStatistik deskriptif setelah scaling (cek min/max):")
        print(df_scaled[numerical_features_to_scale].describe().T)
    else:
        print("Tidak ada fitur numerik asli yang perlu di-scale.")
        df_scaled = df_encoded.copy()
else:
    print("Dataframe df_encoded tidak tersedia.")
    df_scaled = None


In [ ]:
if 'df_scaled' in locals() and df_scaled is not None and 'IMDB_Rating' in df_scaled.columns:
    print('\n---- Manual Pembagian Dataset (Train-Test Split) ----')
    target_column = 'IMDB_Rating'
    X = df_scaled.drop(target_column, axis=1)
    y = df_scaled[target_column]

    print(f"Ukuran Fitur (X) sebelum split: {X.shape}")
    print(f"Ukuran Target (y) sebelum split: {y.shape}")

    try:
        # Cek jumlah kelas unik dan distribusinya
        value_counts = y.value_counts()

        if value_counts.min() < 2 or y.nunique() > 20:
            print("Terdapat kelas dengan jumlah sangat sedikit atau terlalu banyak kelas unik. Menggunakan binning...")

            # Melakukan binning ke dalam 5 kategori berdasarkan quantiles
            y_binned = pd.qcut(y, q=5, labels=False, duplicates='drop')  # duplicates='drop' menghindari error jika batas bin sama

            # Split data dengan stratifikasi berdasarkan bin
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                test_size=0.2,
                random_state=42,
                stratify=y_binned
            )
        else:
            # Lanjutkan split dengan stratifikasi biasa
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                test_size=0.2,
                random_state=42,
                stratify=y
            )

        print("\nUkuran dataset setelah dibagi:")
        print(f"X_train shape: {X_train.shape}")
        print(f"X_test shape: {X_test.shape}")
        print(f"y_train shape: {y_train.shape}")
        print(f"y_test shape: {y_test.shape}")

        print(f"\nProps target (y_train):\n{y_train.value_counts(normalize=True).map('{:.2%}'.format)}")
        print(f"\nProps target (y_test):\n{y_test.value_counts(normalize=True).map('{:.2%}'.format)}")

    except Exception as e:
        print(f"Error saat membagi dataset: {e}")

else:
    print("Dataframe 'df_scaled' atau kolom target 'IMDB_Rating' tidak tersedia.")

**Kesimpulan**

1. Ringkasan Temuan

Dataset yang digunakan merupakan kumpulan data IMDb Top 1000 Movies & TV Shows yang terdiri dari berbagai fitur numerik dan kategorikal.

Fitur numerik: `Runtime`, `IMDB_Rating`, `Meta_score`,`Released_Year`, `No_of_Votes`, `Gross`.

Fitur kategorikal:`Poster_Link`, `Series_Title`, `Genre`, `Certificate`,`Overview`, `Director`, `Star1`, `Star2`, `Star3`, `Star4`.

Dataset ini mengandung sejumlah nilai hilang (missing values), terutama pada kolom `Meta_score` dan `Gross`, yang perlu ditangani sebelum proses analisis lebih lanjut.

2. Ringkasan Persiapan Data
Penanganan Nilai Hilang:
Nilai hilang ditangani dengan metode penghapusan (drop) atau pengisian menggunakan nilai rata-rata/median tergantung pada jenis data.

Tipe Data:
Kolom yang bertipe objek namun mengandung nilai numerik diubah ke tipe numerik (contoh: `Runtime`).

Encoding:
Fitur kategorikal seperti `Genre` dikodekan menggunakan teknik One-Hot Encoding.

Scaling:
Fitur numerik seperti `Runtime` dan `No_of_Votes` diskalakan menggunakan StandardScaler agar memiliki distribusi normal.

Train-Test Split:
Data dibagi menjadi 80% data latih dan 20% data uji.

3. Tantangan

Banyaknya nilai hilang pada fitur penting yang dapat mempengaruhi kualitas model.
Menentukan fitur mana yang paling relevan terhadap variabel target (misalnya `IMDB_Rating`)